In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.6`
import $ivy.`org.apache.spark::spark-core:3.5.6`
import $ivy.`org.postgresql:postgresql:42.7.3`  // Driver JDBC de PostgreSQL
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.{DataFrame, SaveMode}

val spark = SparkSession.builder()
  .appName("PostgreSQL-Notebook")
  .master("local[*]")
  .config("spark.sql.adaptive.enabled", "true")
  .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
  .getOrCreate()

import spark.implicits._

// Configuración de conexión a PostgreSQL
val jdbcUrl = "jdbc:postgresql://localhost:5432/postgres"
val connectionProperties = new java.util.Properties()
connectionProperties.setProperty("user", "postgres")
connectionProperties.setProperty("password", "postgres")
connectionProperties.setProperty("driver", "org.postgresql.Driver")

println("✅ Spark configurado y listo para conectar con PostgreSQL")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/08 22:41:44 INFO SparkContext: Running Spark version 3.5.6
25/06/08 22:41:44 INFO SparkContext: OS info Mac OS X, 15.5, aarch64
25/06/08 22:41:44 INFO SparkContext: Java version 1.8.0_451
25/06/08 22:41:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/08 22:41:44 INFO ResourceUtils: ==============================================================
25/06/08 22:41:44 INFO ResourceUtils: No custom resources configured for spark.driver.
25/06/08 22:41:44 INFO ResourceUtils: ==============================================================
25/06/08 22:41:44 INFO SparkContext: Submitted application: PostgreSQL-Notebook
25/06/08 22:41:44 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHe

✅ Spark configurado y listo para conectar con PostgreSQL


import $ivy.$
import $ivy.$
import $ivy.$
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.{DataFrame, SaveMode}
spark: SparkSession = org.apache.spark.sql.SparkSession@646da869
import spark.implicits._
jdbcUrl: String = "jdbc:postgresql://localhost:5432/postgres"
connectionProperties: java.util.Properties = {user=postgres, password=postgres, driver=org.postgresql.Driver}
res1_9: Object = null
res1_10: Object = null
res1_11: Object = null

In [2]:
// Función para leer tabla desde PostgreSQL
def leerTablaPostgres(tableName: String): DataFrame = {
  spark.read
    .jdbc(jdbcUrl, tableName, connectionProperties)
}

// Función para escribir DataFrame a PostgreSQL
def escribirTablaPostgres(df: DataFrame, tableName: String, mode: SaveMode = SaveMode.Overwrite): Unit = {
  df.write
    .mode(mode)
    .jdbc(jdbcUrl, tableName, connectionProperties)
}

// Función para ejecutar query SQL directamente
def ejecutarQuery(query: String): DataFrame = {
  spark.read
    .format("jdbc")
    .option("url", jdbcUrl)
    .option("query", query)
    .option("user", "postgres")
    .option("password", "postgres")
    .option("driver", "org.postgresql.Driver")
    .load()
}

println("🔧 Funciones de PostgreSQL definidas")

🔧 Funciones de PostgreSQL definidas


defined function leerTablaPostgres
defined function escribirTablaPostgres
defined function ejecutarQuery

In [3]:
// Probar la conexión
try {
  /*
  // Listar todas las tablas
  val tablas = ejecutarQuery("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'
  """)
  
  println("📋 Tablas disponibles en PostgreSQL:")
  tablas.show()
  */
  
  val miTabla = leerTablaPostgres("arbitrios_jm")
  println(s"📊 Registros en la tabla: ${miTabla.count()}")
  miTabla.show(5)
  
} catch {
  case e: Exception => 
    println(s"❌ Error conectando a PostgreSQL: ${e.getMessage}")
    println("🔍 Verifica que PostgreSQL esté ejecutándose y las credenciales sean correctas")
}

25/06/08 22:41:46 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/06/08 22:41:46 INFO SharedState: Warehouse path is 'file:/Users/naodeko/uni/bigdata/scala_notebooks/scala_notebooks/spark-warehouse'.
25/06/08 22:41:47 INFO CodeGenerator: Code generated in 87.904084 ms
25/06/08 22:41:47 INFO DAGScheduler: Registering RDD 2 (count at cmd3.sc:16) as input to shuffle 0
25/06/08 22:41:47 INFO DAGScheduler: Got map stage job 0 (count at cmd3.sc:16) with 1 output partitions
25/06/08 22:41:47 INFO DAGScheduler: Final stage: ShuffleMapStage 0 (count at cmd3.sc:16)
25/06/08 22:41:47 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:47 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:47 INFO DAGScheduler: Submitting ShuffleMapStage 0 (MapPartitionsRDD[2] at count at cmd3.sc:16), which has no missing parents
25/06/08 22:41:47 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 13.8 KiB, f

📊 Registros en la tabla: 220832


25/06/08 22:41:48 INFO CodeGenerator: Code generated in 34.745458 ms
25/06/08 22:41:48 INFO SparkContext: Starting job: show at cmd3.sc:17
25/06/08 22:41:48 INFO DAGScheduler: Got job 2 (show at cmd3.sc:17) with 1 output partitions
25/06/08 22:41:48 INFO DAGScheduler: Final stage: ResultStage 3 (show at cmd3.sc:17)
25/06/08 22:41:48 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:48 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:48 INFO DAGScheduler: Submitting ResultStage 3 (MapPartitionsRDD[8] at show at cmd3.sc:17), which has no missing parents
25/06/08 22:41:48 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 18.9 KiB, free 2004.5 MiB)
25/06/08 22:41:48 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 7.6 KiB, free 2004.5 MiB)
25/06/08 22:41:48 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on 192.168.18.245:57344 (size: 7.6 KiB, free: 2004.6 MiB)
25/06/08 22:41:48 INFO SparkCon

+-----------+-------+--------------------+--------------------+--------------------+---------------------+-------------------+---------------+----------------------+--------------------+-----------------------------+------------------------+-----------------------------------------+----------------------------+------------+---------+----------+---------+
|fecha_corte|periodo|   cod_contribuyente|   nom_contribuyente|          cod_predio|porcentaje_condominio|monto_parque_jardin|monto_serenazgo|monto_residuos_solidos|monto_barrido_calles|descuento_tope__parque_jardin|descuento_tope_serenazgo|descuento_tope_recoleccion_residuo_solido|descuento_tope_barrido_calle|departamento|provincia|  distrito|   ubigeo|
+-----------+-------+--------------------+--------------------+--------------------+---------------------+-------------------+---------------+----------------------+--------------------+-----------------------------+------------------------+-----------------------------------------+---

In [4]:
// PASO 1: Cargar datos y crear vista temporal
try {
  val miTabla = leerTablaPostgres("arbitrios_jm")
  println(s"📊 Registros en la tabla: ${miTabla.count()}")
  
  // Crear vista temporal para usar con spark.sql()
  miTabla.createOrReplaceTempView("arbitrios_jm")
  println("✅ Vista temporal 'arbitrios_jm' creada")
  
  // Importar funciones necesarias
  import org.apache.spark.sql.functions._
  import org.apache.spark.sql.expressions.Window
  
  println("🔧 Funciones importadas y listas para MapReduce")
  
} catch {
  case e: Exception => 
    println(s"❌ Error: ${e.getMessage}")
}

25/06/08 22:41:48 INFO DAGScheduler: Registering RDD 11 (count at cmd4.sc:4) as input to shuffle 1
25/06/08 22:41:48 INFO DAGScheduler: Got map stage job 3 (count at cmd4.sc:4) with 1 output partitions
25/06/08 22:41:48 INFO DAGScheduler: Final stage: ShuffleMapStage 4 (count at cmd4.sc:4)
25/06/08 22:41:48 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:48 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:48 INFO DAGScheduler: Submitting ShuffleMapStage 4 (MapPartitionsRDD[11] at count at cmd4.sc:4), which has no missing parents
25/06/08 22:41:48 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 13.8 KiB, free 2004.6 MiB)
25/06/08 22:41:48 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 7.2 KiB, free 2004.6 MiB)
25/06/08 22:41:48 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on 192.168.18.245:57344 (size: 7.2 KiB, free: 2004.6 MiB)
25/06/08 22:41:48 INFO SparkContext: Created broadca

📊 Registros en la tabla: 220832
✅ Vista temporal 'arbitrios_jm' creada
🔧 Funciones importadas y listas para MapReduce


In [5]:
// CONSULTA 1: MapReduce - Montos totales por contribuyente
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

println("🔍 CONSULTA 1: MapReduce - Montos totales por contribuyente")

val df = spark.table("arbitrios_jm")

// MapReduce 1: Map - Calcular monto total por registro
val mapped1 = df.select(
  col("cod_contribuyente"),
  col("distrito"),
  col("periodo"),
  (col("monto_parque_jardin").cast("double") + 
   col("monto_serenazgo").cast("double") + 
   col("monto_residuos_solidos").cast("double") + 
   col("monto_barrido_calles").cast("double")).as("monto_total_registro"),
  col("porcentaje_condominio").cast("double").as("porcentaje")
).filter(col("monto_parque_jardin").isNotNull)

// Reduce 1: Agrupar por contribuyente
val reduced1 = mapped1.groupBy("cod_contribuyente", "distrito")
  .agg(
    count("*").as("num_predios"),
    sum("monto_total_registro").as("monto_total"),
    avg("porcentaje").as("promedio_condominio"),
    max("periodo").as("ultimo_periodo")
  )
  .orderBy(desc("monto_total"))
  .limit(10)

println("📊 Resultado Consulta 1:")
reduced1.show()

🔍 CONSULTA 1: MapReduce - Montos totales por contribuyente
📊 Resultado Consulta 1:


25/06/08 22:41:49 INFO CodeGenerator: Code generated in 29.585958 ms
25/06/08 22:41:49 INFO DAGScheduler: Registering RDD 17 (show at cmd5.sc:33) as input to shuffle 2
25/06/08 22:41:49 INFO DAGScheduler: Got map stage job 5 (show at cmd5.sc:33) with 1 output partitions
25/06/08 22:41:49 INFO DAGScheduler: Final stage: ShuffleMapStage 7 (show at cmd5.sc:33)
25/06/08 22:41:49 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:49 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:49 INFO DAGScheduler: Submitting ShuffleMapStage 7 (MapPartitionsRDD[17] at show at cmd5.sc:33), which has no missing parents
25/06/08 22:41:49 INFO MemoryStore: Block broadcast_5 stored as values in memory (estimated size 54.5 KiB, free 2004.5 MiB)
25/06/08 22:41:49 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 23.2 KiB, free 2004.5 MiB)
25/06/08 22:41:49 INFO BlockManagerInfo: Added broadcast_5_piece0 in memory on 192.168.18.245:57344 (size: 23.2 KiB,

+--------------------+----------+-----------+------------------+-------------------+--------------+
|   cod_contribuyente|  distrito|num_predios|       monto_total|promedio_condominio|ultimo_periodo|
+--------------------+----------+-----------+------------------+-------------------+--------------+
|55351ada81c0f27f8...|JESUS MARI|        599| 963183.3599999999|              100.0|          2025|
|8b23a5f47f37eaa52...|JESUS MARI|         33|         739913.52|              100.0|          2025|
|39ed9d9016f301c73...|JESUS MARI|        368| 593574.1200000013|              100.0|          2025|
|9e17ffa1e26b269ea...|JESUS MARI|        201| 364319.6400000001|              100.0|          2025|
|c7d7cd94d59a8b17c...|JESUS MARI|        147|339312.60000000003|              100.0|          2025|
|139af841d2e577a63...|JESUS MARI|        414| 320959.0800000003|              100.0|          2025|
|2a0199bb8f20b9035...|JESUS MARI|         19|304931.16000000003|              100.0|          2025|


import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
df: DataFrame = [fecha_corte: int, periodo: int ... 16 more fields]
mapped1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, distrito: string ... 3 more fields]
reduced1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, distrito: string ... 4 more fields]

In [6]:
// CONSULTA 2: MapReduce - Análisis por distrito y período
import org.apache.spark.sql.functions._

println("🔍 CONSULTA 2: MapReduce - Análisis por distrito y período")

val df = spark.table("arbitrios_jm")

// MapReduce 2: Map - Calcular montos con descuentos
val mapped2 = df.select(
  col("distrito"),
  col("periodo").cast("int").as("periodo"),
  col("monto_serenazgo").cast("double").as("monto_serenazgo"),
  col("descuento_tope_serenazgo").cast("double").as("descuento_serenazgo"),
  (col("monto_serenazgo").cast("double") - col("descuento_tope_serenazgo").cast("double")).as("monto_neto_serenazgo"),
  col("porcentaje_condominio").cast("double").as("porcentaje")
).filter(col("monto_serenazgo").isNotNull && col("distrito").isNotNull)

// Reduce 2: Agrupar por distrito y período
val reduced2 = mapped2.groupBy("distrito", "periodo")
  .agg(
    count("*").as("total_predios"),
    round(sum("monto_serenazgo"), 2).as("total_serenazgo_bruto"),
    round(sum("descuento_serenazgo"), 2).as("total_descuentos"),
    round(sum("monto_neto_serenazgo"), 2).as("total_serenazgo_neto"),
    round(avg("porcentaje"), 2).as("promedio_participacion")
  )
  .orderBy("distrito", "periodo")

println("📊 Resultado Consulta 2:")
reduced2.show()

🔍 CONSULTA 2: MapReduce - Análisis por distrito y período
📊 Resultado Consulta 2:


25/06/08 22:41:50 INFO CodeGenerator: Code generated in 21.797042 ms
25/06/08 22:41:50 INFO DAGScheduler: Registering RDD 24 (show at cmd6.sc:30) as input to shuffle 3
25/06/08 22:41:50 INFO DAGScheduler: Got map stage job 7 (show at cmd6.sc:30) with 1 output partitions
25/06/08 22:41:50 INFO DAGScheduler: Final stage: ShuffleMapStage 10 (show at cmd6.sc:30)
25/06/08 22:41:50 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:50 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:50 INFO DAGScheduler: Submitting ShuffleMapStage 10 (MapPartitionsRDD[24] at show at cmd6.sc:30), which has no missing parents
25/06/08 22:41:50 INFO MemoryStore: Block broadcast_7 stored as values in memory (estimated size 56.4 KiB, free 2004.5 MiB)
25/06/08 22:41:50 INFO MemoryStore: Block broadcast_7_piece0 stored as bytes in memory (estimated size 23.4 KiB, free 2004.4 MiB)
25/06/08 22:41:50 INFO BlockManagerInfo: Added broadcast_7_piece0 in memory on 192.168.18.245:57344 (size: 23.4 Ki

+----------+-------+-------------+---------------------+----------------+--------------------+----------------------+
|  distrito|periodo|total_predios|total_serenazgo_bruto|total_descuentos|total_serenazgo_neto|promedio_participacion|
+----------+-------+-------------+---------------------+----------------+--------------------+----------------------+
|JESUS MARI|   2023|        69172|        1.783041608E7|      2477705.57|       1.535271051E7|                 83.74|
|JESUS MARI|   2024|        73329|        1.949122927E7|       637110.38|       1.885411889E7|                 83.14|
|JESUS MARI|   2025|        78294|        2.079256492E7|         85947.0|       2.070661792E7|                 83.03|
+----------+-------+-------------+---------------------+----------------+--------------------+----------------------+



import org.apache.spark.sql.functions._
df: DataFrame = [fecha_corte: int, periodo: int ... 16 more fields]
mapped2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, periodo: int ... 4 more fields]
reduced2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, periodo: int ... 5 more fields]

In [7]:
// CONSULTA 3: MapReduce - Eficiencia de cobro por contribuyente
import org.apache.spark.sql.functions._

println("🔍 CONSULTA 3: MapReduce - Eficiencia de cobro por contribuyente")

val df = spark.table("arbitrios_jm")

// MapReduce 3: Map - Calcular eficiencia por registro
val mapped3 = df.select(
  col("cod_contribuyente"),
  col("distrito"),
  col("monto_parque_jardin").cast("double").as("monto_parque"),
  col("monto_serenazgo").cast("double").as("monto_serenazgo"),
  col("descuento_tope__parque_jardin").cast("double").as("desc_parque"),
  col("descuento_tope_serenazgo").cast("double").as("desc_serenazgo"),
  ((col("monto_parque_jardin").cast("double") + col("monto_serenazgo").cast("double")) - 
   (col("descuento_tope__parque_jardin").cast("double") + col("descuento_tope_serenazgo").cast("double"))).as("monto_neto_total")
).filter(col("monto_parque_jardin").isNotNull && col("monto_serenazgo").isNotNull)

// Reduce 3: Agrupar por contribuyente y calcular eficiencia
val reduced3 = mapped3.groupBy("cod_contribuyente", "distrito")
  .agg(
    count("*").as("num_predios"),
    round(sum("monto_parque"), 2).as("total_parque"),
    round(sum("monto_serenazgo"), 2).as("total_serenazgo"),
    round(sum("desc_parque"), 2).as("total_desc_parque"),
    round(sum("desc_serenazgo"), 2).as("total_desc_serenazgo"),
    round(sum("monto_neto_total"), 2).as("total_neto")
  )
  .filter(col("num_predios") > 1)
  .withColumn("eficiencia_cobro", 
              round(col("total_neto") / (col("total_parque") + col("total_serenazgo")) * 100, 2))
  .orderBy(desc("total_neto"))
  .limit(15)

println("📊 Resultado Consulta 3:")
reduced3.show()

🔍 CONSULTA 3: MapReduce - Eficiencia de cobro por contribuyente
📊 Resultado Consulta 3:


25/06/08 22:41:51 INFO CodeGenerator: Code generated in 11.542375 ms
25/06/08 22:41:51 INFO DAGScheduler: Registering RDD 31 (show at cmd7.sc:37) as input to shuffle 4
25/06/08 22:41:51 INFO DAGScheduler: Got map stage job 9 (show at cmd7.sc:37) with 1 output partitions
25/06/08 22:41:51 INFO DAGScheduler: Final stage: ShuffleMapStage 13 (show at cmd7.sc:37)
25/06/08 22:41:51 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:51 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:51 INFO DAGScheduler: Submitting ShuffleMapStage 13 (MapPartitionsRDD[31] at show at cmd7.sc:37), which has no missing parents
25/06/08 22:41:51 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 59.8 KiB, free 2004.4 MiB)
25/06/08 22:41:51 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 24.0 KiB, free 2004.4 MiB)
25/06/08 22:41:51 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on 192.168.18.245:57344 (size: 24.0 Ki

+--------------------+----------+-----------+------------+---------------+-----------------+--------------------+----------+----------------+
|   cod_contribuyente|  distrito|num_predios|total_parque|total_serenazgo|total_desc_parque|total_desc_serenazgo|total_neto|eficiencia_cobro|
+--------------------+----------+-----------+------------+---------------+-----------------+--------------------+----------+----------------+
|55351ada81c0f27f8...|JESUS MARI|        599|   160163.16|      657369.24|              0.0|            70384.32| 747148.08|           91.39|
|8b23a5f47f37eaa52...|JESUS MARI|         33|     9724.32|       709412.4|              0.0|             79134.6| 640002.12|            89.0|
|9e17ffa1e26b269ea...|JESUS MARI|        201|    53996.64|       283911.6|              0.0|            31641.72| 306266.52|           90.64|
|c7d7cd94d59a8b17c...|JESUS MARI|        147|    38205.72|      275865.72|              0.0|             23238.0| 290833.44|            92.6|
|2119b

import org.apache.spark.sql.functions._
df: DataFrame = [fecha_corte: int, periodo: int ... 16 more fields]
mapped3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, distrito: string ... 5 more fields]
reduced3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, distrito: string ... 7 more fields]

In [8]:
// AGRUPAR POR TIPOS Y ENCONTRAR MAYORES Y MENORES
import org.apache.spark.sql.functions._

println("📊 ANÁLISIS MapReduce: Agrupación por distrito con extremos")

val df = spark.table("arbitrios_jm")

// Map: Preparar datos por distrito
val mappedDistritos = df.select(
  col("distrito"),
  col("monto_serenazgo").cast("double").as("serenazgo"),
  col("monto_parque_jardin").cast("double").as("parque"),
  col("monto_residuos_solidos").cast("double").as("residuos"),
  col("porcentaje_condominio").cast("double").as("porcentaje")
).filter(col("distrito").isNotNull && 
         col("monto_serenazgo").isNotNull && 
         col("monto_parque_jardin").isNotNull)

// Reduce: Encontrar máximos y mínimos por distrito
val reducedDistritos = mappedDistritos.groupBy("distrito")
  .agg(
    count("*").as("total_registros"),
    round(max("serenazgo"), 2).as("max_serenazgo"),
    round(min("serenazgo"), 2).as("min_serenazgo"),
    round(max("parque"), 2).as("max_parque"),
    round(min("parque"), 2).as("min_parque"),
    round(max("residuos"), 2).as("max_residuos"),
    round(min("residuos"), 2).as("min_residuos"),
    round(max("porcentaje"), 2).as("max_porcentaje"),
    round(min("porcentaje"), 2).as("min_porcentaje"),
    round(avg("serenazgo"), 2).as("promedio_serenazgo")
  )
  .orderBy(desc("promedio_serenazgo"))

println("📊 Máximos y mínimos por distrito:")
reducedDistritos.show()

📊 ANÁLISIS MapReduce: Agrupación por distrito con extremos
📊 Máximos y mínimos por distrito:


25/06/08 22:41:52 INFO CodeGenerator: Code generated in 9.605125 ms
25/06/08 22:41:52 INFO DAGScheduler: Registering RDD 38 (show at cmd8.sc:36) as input to shuffle 5
25/06/08 22:41:52 INFO DAGScheduler: Got map stage job 11 (show at cmd8.sc:36) with 1 output partitions
25/06/08 22:41:52 INFO DAGScheduler: Final stage: ShuffleMapStage 16 (show at cmd8.sc:36)
25/06/08 22:41:52 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:52 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:52 INFO DAGScheduler: Submitting ShuffleMapStage 16 (MapPartitionsRDD[38] at show at cmd8.sc:36), which has no missing parents
25/06/08 22:41:52 INFO MemoryStore: Block broadcast_11 stored as values in memory (estimated size 62.5 KiB, free 2004.4 MiB)
25/06/08 22:41:52 INFO MemoryStore: Block broadcast_11_piece0 stored as bytes in memory (estimated size 24.4 KiB, free 2004.3 MiB)
25/06/08 22:41:52 INFO BlockManagerInfo: Added broadcast_11_piece0 in memory on 192.168.18.245:57344 (size: 24.4

+----------+---------------+-------------+-------------+----------+----------+------------+------------+--------------+--------------+------------------+
|  distrito|total_registros|max_serenazgo|min_serenazgo|max_parque|min_parque|max_residuos|min_residuos|max_porcentaje|min_porcentaje|promedio_serenazgo|
+----------+---------------+-------------+-------------+----------+----------+------------+------------+--------------+--------------+------------------+
|JESUS MARI|         220795|     27102.12|          0.0|    308.64|       0.0|      998.04|         0.0|         100.0|          0.16|             263.2|
+----------+---------------+-------------+-------------+----------+----------+------------+------------+--------------+--------------+------------------+



import org.apache.spark.sql.functions._
df: DataFrame = [fecha_corte: int, periodo: int ... 16 more fields]
mappedDistritos: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, serenazgo: double ... 3 more fields]
reducedDistritos: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, total_registros: bigint ... 9 more fields]

In [9]:
// ESTADÍSTICAS DESCRIPTIVAS
import org.apache.spark.sql.functions._

println("📊 ESTADÍSTICAS DESCRIPTIVAS MapReduce para MONTO_RESIDUOS_SOLIDOS")

val df = spark.table("arbitrios_jm")

// Map: Preparar datos para estadísticas
val mappedEstadisticas = df.select(
  col("monto_residuos_solidos").cast("double").as("residuos")
).filter(col("monto_residuos_solidos").isNotNull)

// Reduce 1: Estadísticas básicas
val estadisticasBasicas = mappedEstadisticas.agg(
  count("residuos").as("total_registros"),
  round(avg("residuos"), 2).as("promedio"),
  round(stddev("residuos"), 2).as("desviacion_estandar"),
  round(max("residuos"), 2).as("maximo"),
  round(min("residuos"), 2).as("minimo"),
  expr("percentile_approx(residuos, 0.5)").as("mediana_aproximada")
)

println("📊 Estadísticas generales:")
estadisticasBasicas.show()

📊 ESTADÍSTICAS DESCRIPTIVAS MapReduce para MONTO_RESIDUOS_SOLIDOS
📊 Estadísticas generales:


25/06/08 22:41:52 INFO CodeGenerator: Code generated in 1.926042 ms
25/06/08 22:41:52 INFO DAGScheduler: Registering RDD 45 (show at cmd9.sc:24) as input to shuffle 6
25/06/08 22:41:52 INFO DAGScheduler: Got map stage job 13 (show at cmd9.sc:24) with 1 output partitions
25/06/08 22:41:52 INFO DAGScheduler: Final stage: ShuffleMapStage 19 (show at cmd9.sc:24)
25/06/08 22:41:52 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:52 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:52 INFO DAGScheduler: Submitting ShuffleMapStage 19 (MapPartitionsRDD[45] at show at cmd9.sc:24), which has no missing parents
25/06/08 22:41:52 INFO MemoryStore: Block broadcast_13 stored as values in memory (estimated size 34.1 KiB, free 2004.4 MiB)
25/06/08 22:41:52 INFO MemoryStore: Block broadcast_13_piece0 stored as bytes in memory (estimated size 16.2 KiB, free 2004.4 MiB)
25/06/08 22:41:52 INFO BlockManagerInfo: Added broadcast_13_piece0 in memory on 192.168.18.245:57344 (size: 16.2

+---------------+--------+-------------------+------+------+------------------+
|total_registros|promedio|desviacion_estandar|maximo|minimo|mediana_aproximada|
+---------------+--------+-------------------+------+------+------------------+
|         220795|   57.98|              80.43|998.04|   0.0|             39.96|
+---------------+--------+-------------------+------+------+------------------+



import org.apache.spark.sql.functions._
df: DataFrame = [fecha_corte: int, periodo: int ... 16 more fields]
mappedEstadisticas: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [residuos: double]
estadisticasBasicas: DataFrame = [total_registros: bigint, promedio: double ... 4 more fields]

In [10]:
// Map-Reduce por distrito
println("📊 ESTADÍSTICAS por DISTRITO:")

val mappedPorDistrito = df.select(
  col("distrito"),
  col("monto_residuos_solidos").cast("double").as("residuos")
).filter(col("distrito").isNotNull && col("monto_residuos_solidos").isNotNull)

val estadisticasPorDistrito = mappedPorDistrito.groupBy("distrito")
  .agg(
    count("residuos").as("registros"),
    round(avg("residuos"), 2).as("promedio"),
    round(stddev("residuos"), 2).as("desviacion"),
    round(max("residuos"), 2).as("maximo"),
    round(min("residuos"), 2).as("minimo"),
    expr("percentile_approx(residuos, 0.5)").as("mediana")
  )
  .filter(col("registros") > 100)
  .orderBy(desc("promedio"))

println("📊 Estadísticas por distrito:")
estadisticasPorDistrito.show()

25/06/08 22:41:52 INFO BlockManagerInfo: Removed broadcast_13_piece0 on 192.168.18.245:57344 in memory (size: 16.2 KiB, free: 2004.5 MiB)
25/06/08 22:41:52 INFO BlockManagerInfo: Removed broadcast_11_piece0 on 192.168.18.245:57344 in memory (size: 24.4 KiB, free: 2004.6 MiB)
25/06/08 22:41:52 INFO BlockManagerInfo: Removed broadcast_12_piece0 on 192.168.18.245:57344 in memory (size: 30.0 KiB, free: 2004.6 MiB)
25/06/08 22:41:52 INFO BlockManagerInfo: Removed broadcast_14_piece0 on 192.168.18.245:57344 in memory (size: 19.7 KiB, free: 2004.6 MiB)


📊 ESTADÍSTICAS por DISTRITO:
📊 Estadísticas por distrito:


25/06/08 22:41:53 INFO CodeGenerator: Code generated in 2.111709 ms
25/06/08 22:41:53 INFO DAGScheduler: Registering RDD 52 (show at cmd10.sc:22) as input to shuffle 7
25/06/08 22:41:53 INFO DAGScheduler: Got map stage job 15 (show at cmd10.sc:22) with 1 output partitions
25/06/08 22:41:53 INFO DAGScheduler: Final stage: ShuffleMapStage 22 (show at cmd10.sc:22)
25/06/08 22:41:53 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:53 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:53 INFO DAGScheduler: Submitting ShuffleMapStage 22 (MapPartitionsRDD[52] at show at cmd10.sc:22), which has no missing parents
25/06/08 22:41:53 INFO MemoryStore: Block broadcast_15 stored as values in memory (estimated size 35.0 KiB, free 2004.6 MiB)
25/06/08 22:41:53 INFO MemoryStore: Block broadcast_15_piece0 stored as bytes in memory (estimated size 16.5 KiB, free 2004.5 MiB)
25/06/08 22:41:53 INFO BlockManagerInfo: Added broadcast_15_piece0 in memory on 192.168.18.245:57344 (size: 

+----------+---------+--------+----------+------+------+-------+
|  distrito|registros|promedio|desviacion|maximo|minimo|mediana|
+----------+---------+--------+----------+------+------+-------+
|JESUS MARI|   220795|   57.98|     80.43|998.04|   0.0|  39.96|
+----------+---------+--------+----------+------+------+-------+



mappedPorDistrito: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, residuos: double]
estadisticasPorDistrito: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, registros: bigint ... 5 more fields]

In [11]:
// CONSULTA DECIMAL 1: Análisis de ratios complejos (3 MapReduce)
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

println("💰 CONSULTA DECIMAL 1: Análisis de ratios complejos (3 MapReduce)")

val df = spark.table("arbitrios_jm")

// MapReduce 1: Calcular ratios básicos por registro
val mapReduce1_Ratios = df.select(
  col("distrito"),
  col("cod_contribuyente"),
  col("monto_serenazgo").cast("double").as("serenazgo"),
  col("monto_parque_jardin").cast("double").as("parque"),
  col("monto_residuos_solidos").cast("double").as("residuos"),
  col("porcentaje_condominio").cast("double").as("porcentaje")
).filter(col("monto_serenazgo").isNotNull && 
         col("monto_parque_jardin").isNotNull && 
         col("distrito").isNotNull)
.withColumn("ratio_serenazgo_parque", 
            round(when(col("parque") =!= 0, col("serenazgo") / col("parque")).otherwise(null), 4))
.withColumn("ratio_residuos_serenazgo", 
            round(when(col("serenazgo") =!= 0, col("residuos") / col("serenazgo")).otherwise(null), 4))

// MapReduce 2: Agrupar por distrito y calcular promedios
val mapReduce2_Agrupado = mapReduce1_Ratios.groupBy("distrito")
  .agg(
    count("*").as("total_predios"),
    round(avg("ratio_serenazgo_parque"), 4).as("ratio_promedio_ser_par"),
    round(avg("ratio_residuos_serenazgo"), 4).as("ratio_promedio_res_ser"),
    round(stddev("ratio_serenazgo_parque"), 4).as("desviacion_ratio"),
    round(avg("porcentaje"), 2).as("promedio_participacion")
  )
  .filter(col("total_predios") > 50)

// MapReduce 3: Calcular ranking y métricas finales
val mapReduce3_Final = mapReduce2_Agrupado
  .withColumn("indice_complejidad", 
              round(col("ratio_promedio_ser_par") * col("ratio_promedio_res_ser") * col("desviacion_ratio"), 6))
  .withColumn("ranking", 
              row_number().over(Window.orderBy(desc("indice_complejidad"))))
  .orderBy("ranking")

println("📊 Resultado Consulta Decimal 1:")
mapReduce3_Final.show()

💰 CONSULTA DECIMAL 1: Análisis de ratios complejos (3 MapReduce)
📊 Resultado Consulta Decimal 1:


25/06/08 22:41:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/08 22:41:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/08 22:41:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/08 22:41:53 INFO CodeGenerator: Code generated in 10.798458 ms
25/06/08 22:41:53 INFO DAGScheduler: Registering RDD 59 (show at cmd11.sc:45) as input to shuffle 8
25/06/08 22:41:53 INFO DAGScheduler: Got map stage job 17 (show at cmd11.sc:45) with 1 output partitions
25/06/08 22:41:53 INFO DAGScheduler: Final stage: ShuffleMapStage 25 (show at cmd11.sc:45)
25/06/08 22:41:53 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:53 INFO DAGScheduler: Missing parents: List()
25/06/08 22

+----------+-------------+----------------------+----------------------+----------------+----------------------+------------------+-------+
|  distrito|total_predios|ratio_promedio_ser_par|ratio_promedio_res_ser|desviacion_ratio|promedio_participacion|indice_complejidad|ranking|
+----------+-------------+----------------------+----------------------+----------------+----------------------+------------------+-------+
|JESUS MARI|       220795|                1.7186|                0.2727|          4.1802|                 83.29|          1.959102|      1|
+----------+-------------+----------------------+----------------------+----------------+----------------------+------------------+-------+



import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
df: DataFrame = [fecha_corte: int, periodo: int ... 16 more fields]
mapReduce1_Ratios: DataFrame = [distrito: string, cod_contribuyente: string ... 6 more fields]
mapReduce2_Agrupado: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, total_predios: bigint ... 4 more fields]
mapReduce3_Final: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, total_predios: bigint ... 6 more fields]

In [12]:
// CONSULTA DECIMAL 2: Análisis de concentración financiera (3 MapReduce)
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

println("💰 CONSULTA DECIMAL 2: Análisis de concentración financiera (3 MapReduce)")

val df = spark.table("arbitrios_jm")

// MapReduce 1: Calcular monto total por contribuyente
val mapReduce1_Concentracion = df.select(
  col("cod_contribuyente"),
  col("distrito"),
  (col("monto_parque_jardin").cast("double") + 
   col("monto_serenazgo").cast("double") + 
   col("monto_residuos_solidos").cast("double") + 
   col("monto_barrido_calles").cast("double")).as("monto_total_contrib"),
  col("porcentaje_condominio").cast("double").as("participacion")
).filter(col("monto_parque_jardin").isNotNull)

// MapReduce 2: Agrupar por contribuyente y calcular totales
val mapReduce2_TotalesContrib = mapReduce1_Concentracion.groupBy("cod_contribuyente", "distrito")
  .agg(
    sum("monto_total_contrib").as("total_contribuyente"),
    avg("participacion").as("participacion_promedio"),
    count("*").as("num_predios")
  )
  .filter(col("num_predios") > 0)

// MapReduce 3: Calcular percentiles y concentración
val totalGeneral = mapReduce2_TotalesContrib.agg(sum("total_contribuyente")).collect()(0)(0).asInstanceOf[Double]

val mapReduce3_Concentracion = mapReduce2_TotalesContrib
  .withColumn("participacion_total_pct", 
              round(col("total_contribuyente") / lit(totalGeneral) * 100, 4))
  .withColumn("decil", 
              ntile(10).over(Window.orderBy(desc("total_contribuyente"))))
  .orderBy(desc("total_contribuyente"))
  .limit(20)

println("📊 Resultado Consulta Decimal 2:")
mapReduce3_Concentracion.show()

💰 CONSULTA DECIMAL 2: Análisis de concentración financiera (3 MapReduce)


25/06/08 22:41:54 INFO CodeGenerator: Code generated in 8.897625 ms
25/06/08 22:41:54 INFO DAGScheduler: Registering RDD 70 (collect at cmd12.sc:30) as input to shuffle 10
25/06/08 22:41:54 INFO DAGScheduler: Got map stage job 20 (collect at cmd12.sc:30) with 1 output partitions
25/06/08 22:41:54 INFO DAGScheduler: Final stage: ShuffleMapStage 31 (collect at cmd12.sc:30)
25/06/08 22:41:54 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:54 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:54 INFO DAGScheduler: Submitting ShuffleMapStage 31 (MapPartitionsRDD[70] at collect at cmd12.sc:30), which has no missing parents
25/06/08 22:41:54 INFO MemoryStore: Block broadcast_20 stored as values in memory (estimated size 46.5 KiB, free 2004.3 MiB)
25/06/08 22:41:54 INFO MemoryStore: Block broadcast_20_piece0 stored as bytes in memory (estimated size 21.0 KiB, free 2004.2 MiB)
25/06/08 22:41:54 INFO BlockManagerInfo: Added broadcast_20_piece0 in memory on 192.168.18.245:

📊 Resultado Consulta Decimal 2:


25/06/08 22:41:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/08 22:41:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/08 22:41:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/08 22:41:54 INFO CodeGenerator: Code generated in 7.854208 ms
25/06/08 22:41:54 INFO DAGScheduler: Registering RDD 79 (show at cmd12.sc:41) as input to shuffle 12
25/06/08 22:41:54 INFO DAGScheduler: Got map stage job 23 (show at cmd12.sc:41) with 1 output partitions
25/06/08 22:41:54 INFO DAGScheduler: Final stage: ShuffleMapStage 37 (show at cmd12.sc:41)
25/06/08 22:41:54 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:54 INFO DAGScheduler: Missing parents: List()
25/06/08 22

+--------------------+----------+-------------------+----------------------+-----------+-----------------------+-----+
|   cod_contribuyente|  distrito|total_contribuyente|participacion_promedio|num_predios|participacion_total_pct|decil|
+--------------------+----------+-------------------+----------------------+-----------+-----------------------+-----+
|55351ada81c0f27f8...|JESUS MARI|  963183.3599999999|                 100.0|        599|                 0.8545|    1|
|8b23a5f47f37eaa52...|JESUS MARI|          739913.52|                 100.0|         33|                 0.6564|    1|
|39ed9d9016f301c73...|JESUS MARI|  593574.1200000013|                 100.0|        368|                 0.5266|    1|
|9e17ffa1e26b269ea...|JESUS MARI|  364319.6400000001|                 100.0|        201|                 0.3232|    1|
|c7d7cd94d59a8b17c...|JESUS MARI| 339312.60000000003|                 100.0|        147|                  0.301|    1|
|139af841d2e577a63...|JESUS MARI|  320959.080000

import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
df: DataFrame = [fecha_corte: int, periodo: int ... 16 more fields]
mapReduce1_Concentracion: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, distrito: string ... 2 more fields]
mapReduce2_TotalesContrib: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, distrito: string ... 3 more fields]
totalGeneral: Double = 1.127245111600002E8
mapReduce3_Concentracion: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, distrito: string ... 5 more fields]

In [13]:
// CONSULTA DECIMAL 3: Análisis de eficiencia temporal (3 MapReduce)
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

println("💰 CONSULTA DECIMAL 3: Análisis de eficiencia temporal (3 MapReduce)")

val df = spark.table("arbitrios_jm")

// MapReduce 1: Calcular eficiencia por período y registro
val mapReduce1_Eficiencia = df.select(
  col("periodo").cast("int").as("periodo"),
  col("distrito"),
  col("monto_serenazgo").cast("double").as("monto_bruto"),
  col("descuento_tope_serenazgo").cast("double").as("descuento"),
  col("porcentaje_condominio").cast("double").as("participacion")
).filter(col("periodo").isNotNull && col("monto_serenazgo").isNotNull)
.withColumn("monto_neto", col("monto_bruto") - col("descuento"))
.withColumn("eficiencia_cobro", 
            round(when(col("monto_bruto") =!= 0, col("monto_neto") / col("monto_bruto") * 100).otherwise(0), 2))

// MapReduce 2: Agrupar por período y distrito
val mapReduce2_EficienciaPeriodo = mapReduce1_Eficiencia.groupBy("periodo", "distrito")
  .agg(
    count("*").as("registros"),
    round(avg("eficiencia_cobro"), 2).as("eficiencia_promedio"),
    round(stddev("eficiencia_cobro"), 2).as("variabilidad_eficiencia"),
    sum("monto_bruto").as("total_bruto"),
    sum("monto_neto").as("total_neto")
  )
  .filter(col("registros") > 10)

// MapReduce 3: Calcular métricas comparativas y tendencias
val mapReduce3_Tendencias = mapReduce2_EficienciaPeriodo
  .withColumn("ratio_neto_bruto", 
              round(when(col("total_bruto") =!= 0, col("total_neto") / col("total_bruto")).otherwise(0), 4))
  .withColumn("score_eficiencia", 
              round(when(col("variabilidad_eficiencia") =!= 0, 
                         col("eficiencia_promedio") * col("ratio_neto_bruto") / col("variabilidad_eficiencia"))
                    .otherwise(0), 4))
  .orderBy(desc("score_eficiencia"))

println("📊 Resultado Consulta Decimal 3:")
mapReduce3_Tendencias.show()

💰 CONSULTA DECIMAL 3: Análisis de eficiencia temporal (3 MapReduce)
📊 Resultado Consulta Decimal 3:


25/06/08 22:41:55 INFO CodeGenerator: Code generated in 9.290125 ms
25/06/08 22:41:55 INFO DAGScheduler: Registering RDD 90 (show at cmd13.sc:43) as input to shuffle 14
25/06/08 22:41:55 INFO DAGScheduler: Got map stage job 26 (show at cmd13.sc:43) with 1 output partitions
25/06/08 22:41:55 INFO DAGScheduler: Final stage: ShuffleMapStage 43 (show at cmd13.sc:43)
25/06/08 22:41:55 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:55 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:55 INFO DAGScheduler: Submitting ShuffleMapStage 43 (MapPartitionsRDD[90] at show at cmd13.sc:43), which has no missing parents
25/06/08 22:41:55 INFO MemoryStore: Block broadcast_26 stored as values in memory (estimated size 63.8 KiB, free 2004.3 MiB)
25/06/08 22:41:55 INFO MemoryStore: Block broadcast_26_piece0 stored as bytes in memory (estimated size 26.1 KiB, free 2004.3 MiB)
25/06/08 22:41:55 INFO BlockManagerInfo: Added broadcast_26_piece0 in memory on 192.168.18.245:57344 (size:

+-------+----------+---------+-------------------+-----------------------+--------------------+--------------------+----------------+----------------+
|periodo|  distrito|registros|eficiencia_promedio|variabilidad_eficiencia|         total_bruto|          total_neto|ratio_neto_bruto|score_eficiencia|
+-------+----------+---------+-------------------+-----------------------+--------------------+--------------------+----------------+----------------+
|   2025|JESUS MARI|    78294|              64.94|                  47.63|2.0792564919987515E7|2.0706617919987597E7|          0.9959|          1.3578|
|   2024|JESUS MARI|    73329|              63.93|                  46.48| 1.949122926999415E7|1.8854118889998067E7|          0.9673|          1.3305|
|   2023|JESUS MARI|    69172|               57.3|                  41.83| 1.783041607999438E7|1.5352710509994695E7|           0.861|          1.1794|
+-------+----------+---------+-------------------+-----------------------+--------------------

import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
df: DataFrame = [fecha_corte: int, periodo: int ... 16 more fields]
mapReduce1_Eficiencia: DataFrame = [periodo: int, distrito: string ... 5 more fields]
mapReduce2_EficienciaPeriodo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [periodo: int, distrito: string ... 5 more fields]
mapReduce3_Tendencias: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [periodo: int, distrito: string ... 7 more fields]

In [14]:
// 1. MOSTRAR COLUMNAS ESPECÍFICAS DE LAS TABLAS
import org.apache.spark.sql.functions._

println("📋 1. MOSTRAR COLUMNAS ESPECÍFICAS")

val df = spark.table("arbitrios_jm")

// Seleccionar columnas específicas
val columnasEspecificas = df.select(
  col("cod_contribuyente"), 
  col("distrito"), 
  col("periodo"),
  col("monto_serenazgo"),
  col("monto_parque_jardin")
)

println("📊 Columnas específicas seleccionadas:")
columnasEspecificas.show(10)

📋 1. MOSTRAR COLUMNAS ESPECÍFICAS
📊 Columnas específicas seleccionadas:


25/06/08 22:41:56 INFO CodeGenerator: Code generated in 3.770292 ms
25/06/08 22:41:56 INFO SparkContext: Starting job: show at cmd14.sc:18
25/06/08 22:41:56 INFO DAGScheduler: Got job 28 (show at cmd14.sc:18) with 1 output partitions
25/06/08 22:41:56 INFO DAGScheduler: Final stage: ResultStage 46 (show at cmd14.sc:18)
25/06/08 22:41:56 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:56 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:56 INFO DAGScheduler: Submitting ResultStage 46 (MapPartitionsRDD[97] at show at cmd14.sc:18), which has no missing parents
25/06/08 22:41:56 INFO MemoryStore: Block broadcast_28 stored as values in memory (estimated size 12.9 KiB, free 2004.4 MiB)
25/06/08 22:41:56 INFO MemoryStore: Block broadcast_28_piece0 stored as bytes in memory (estimated size 6.4 KiB, free 2004.4 MiB)
25/06/08 22:41:56 INFO BlockManagerInfo: Added broadcast_28_piece0 in memory on 192.168.18.245:57344 (size: 6.4 KiB, free: 2004.5 MiB)
25/06/08 22:41:56 INF

+--------------------+----------+-------+---------------+-------------------+
|   cod_contribuyente|  distrito|periodo|monto_serenazgo|monto_parque_jardin|
+--------------------+----------+-------+---------------+-------------------+
|1185ae73bff8b13e1...|JESUS MARI|   2025|         333.36|             272.40|
|8d8953f65b09e4c51...|JESUS MARI|   2025|         333.36|             272.40|
|4ae0fb6f97d77457c...|JESUS MARI|   2025|         333.36|             272.40|
|c2b54eb47f53d21f8...|JESUS MARI|   2025|         333.36|             272.40|
|23e90633bf8fc6a60...|JESUS MARI|   2025|         333.36|             272.40|
|23e90633bf8fc6a60...|JESUS MARI|   2025|           0.00|               0.00|
|358a3d5cd904ff2c1...|JESUS MARI|   2025|         333.36|             272.40|
|fb88c78fcbaa2f891...|JESUS MARI|   2025|         333.36|             272.40|
|8ee43da603a07ac3b...|JESUS MARI|   2025|           0.00|               0.00|
|8ee43da603a07ac3b...|JESUS MARI|   2025|           0.00|       

import org.apache.spark.sql.functions._
df: DataFrame = [fecha_corte: int, periodo: int ... 16 more fields]
columnasEspecificas: DataFrame = [cod_contribuyente: string, distrito: string ... 3 more fields]

In [15]:
// 2. UTILIZAR EL COMANDO FILTER
println("🔍 2. UTILIZAR COMANDO FILTER")

// Filter por distrito y montos mayores a cierto valor
val filtradoPorDistrito = df.filter(
  col("distrito") === "JESUS MARIA" && 
  col("monto_serenazgo").cast("double") > 100
)

println("📊 Registros filtrados por distrito JESUS MARIA y monto_serenazgo > 100:")
filtradoPorDistrito.select("cod_contribuyente", "distrito", "monto_serenazgo", "periodo").show(10)

// Filter con múltiples condiciones
val filtroComplejo = df.filter(
  col("periodo").cast("int") >= 2020 && 
  col("monto_parque_jardin").cast("double") > 50 &&
  col("distrito").isNotNull
)

println("📊 Registros filtrados con condiciones múltiples:")
filtroComplejo.select("distrito", "periodo", "monto_parque_jardin").show(10)

🔍 2. UTILIZAR COMANDO FILTER
📊 Registros filtrados por distrito JESUS MARIA y monto_serenazgo > 100:


25/06/08 22:41:56 INFO CodeGenerator: Code generated in 4.510542 ms
25/06/08 22:41:56 INFO SparkContext: Starting job: show at cmd15.sc:11
25/06/08 22:41:56 INFO DAGScheduler: Got job 29 (show at cmd15.sc:11) with 1 output partitions
25/06/08 22:41:56 INFO DAGScheduler: Final stage: ResultStage 47 (show at cmd15.sc:11)
25/06/08 22:41:56 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:56 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:56 INFO DAGScheduler: Submitting ResultStage 47 (MapPartitionsRDD[100] at show at cmd15.sc:11), which has no missing parents
25/06/08 22:41:56 INFO MemoryStore: Block broadcast_29 stored as values in memory (estimated size 14.3 KiB, free 2004.4 MiB)
25/06/08 22:41:56 INFO MemoryStore: Block broadcast_29_piece0 stored as bytes in memory (estimated size 7.1 KiB, free 2004.4 MiB)
25/06/08 22:41:56 INFO BlockManagerInfo: Added broadcast_29_piece0 in memory on 192.168.18.245:57344 (size: 7.1 KiB, free: 2004.5 MiB)
25/06/08 22:41:56 IN

+-----------------+--------+---------------+-------+
|cod_contribuyente|distrito|monto_serenazgo|periodo|
+-----------------+--------+---------------+-------+
+-----------------+--------+---------------+-------+

📊 Registros filtrados con condiciones múltiples:


25/06/08 22:41:56 INFO CodeGenerator: Code generated in 3.768958 ms
25/06/08 22:41:56 INFO SparkContext: Starting job: show at cmd15.sc:21
25/06/08 22:41:56 INFO DAGScheduler: Got job 30 (show at cmd15.sc:21) with 1 output partitions
25/06/08 22:41:56 INFO DAGScheduler: Final stage: ResultStage 48 (show at cmd15.sc:21)
25/06/08 22:41:56 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:56 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:56 INFO DAGScheduler: Submitting ResultStage 48 (MapPartitionsRDD[103] at show at cmd15.sc:21), which has no missing parents
25/06/08 22:41:56 INFO MemoryStore: Block broadcast_30 stored as values in memory (estimated size 13.9 KiB, free 2004.4 MiB)
25/06/08 22:41:56 INFO MemoryStore: Block broadcast_30_piece0 stored as bytes in memory (estimated size 6.9 KiB, free 2004.3 MiB)
25/06/08 22:41:56 INFO BlockManagerInfo: Added broadcast_30_piece0 in memory on 192.168.18.245:57344 (size: 6.9 KiB, free: 2004.5 MiB)
25/06/08 22:41:56 IN

+----------+-------+-------------------+
|  distrito|periodo|monto_parque_jardin|
+----------+-------+-------------------+
|JESUS MARI|   2025|             272.40|
|JESUS MARI|   2025|             272.40|
|JESUS MARI|   2025|             272.40|
|JESUS MARI|   2025|             272.40|
|JESUS MARI|   2025|             272.40|
|JESUS MARI|   2025|             272.40|
|JESUS MARI|   2025|             272.40|
|JESUS MARI|   2025|             136.20|
|JESUS MARI|   2025|             136.20|
|JESUS MARI|   2025|             272.40|
+----------+-------+-------------------+
only showing top 10 rows



filtradoPorDistrito: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [fecha_corte: int, periodo: int ... 16 more fields]
filtroComplejo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [fecha_corte: int, periodo: int ... 16 more fields]

In [16]:
// 3. MOSTRAR INFORMACIÓN ORDENADA
println("📈 3. MOSTRAR INFORMACIÓN ORDENADA")

// Ordenar por monto_serenazgo descendente
val ordenadoPorMonto = df.select(
  col("cod_contribuyente"),
  col("distrito"),
  col("monto_serenazgo").cast("double").as("monto_serenazgo")
).orderBy(desc("monto_serenazgo"))

println("📊 Registros ordenados por monto_serenazgo (descendente):")
ordenadoPorMonto.show(15)

// Ordenar por múltiples columnas
val ordenMultiple = df.select(
  col("distrito"),
  col("periodo").cast("int").as("periodo"),
  col("monto_parque_jardin").cast("double").as("monto_parque")
).filter(
  col("distrito").isNotNull && 
  col("periodo").isNotNull && 
  col("monto_parque_jardin").isNotNull
).dropDuplicates()
.orderBy(asc("distrito"), desc("periodo"), desc("monto_parque"))

println("📊 Registros ordenados por distrito (asc), período (desc), monto_parque (desc):")
ordenMultiple.show(15)

📈 3. MOSTRAR INFORMACIÓN ORDENADA
📊 Registros ordenados por monto_serenazgo (descendente):


25/06/08 22:41:56 INFO CodeGenerator: Code generated in 2.234708 ms
25/06/08 22:41:56 INFO SparkContext: Starting job: show at cmd16.sc:12
25/06/08 22:41:56 INFO DAGScheduler: Got job 31 (show at cmd16.sc:12) with 1 output partitions
25/06/08 22:41:56 INFO DAGScheduler: Final stage: ResultStage 49 (show at cmd16.sc:12)
25/06/08 22:41:56 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:56 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:56 INFO DAGScheduler: Submitting ResultStage 49 (MapPartitionsRDD[107] at show at cmd16.sc:12), which has no missing parents
25/06/08 22:41:56 INFO MemoryStore: Block broadcast_31 stored as values in memory (estimated size 14.4 KiB, free 2004.3 MiB)
25/06/08 22:41:56 INFO MemoryStore: Block broadcast_31_piece0 stored as bytes in memory (estimated size 7.1 KiB, free 2004.3 MiB)
25/06/08 22:41:56 INFO BlockManagerInfo: Added broadcast_31_piece0 in memory on 192.168.18.245:57344 (size: 7.1 KiB, free: 2004.5 MiB)
25/06/08 22:41:56 IN

+--------------------+----------+---------------+
|   cod_contribuyente|  distrito|monto_serenazgo|
+--------------------+----------+---------------+
|539811369851406ad...|JESUS MARI|       27102.12|
|904441682e97b7666...|JESUS MARI|       27102.12|
|8b23a5f47f37eaa52...|JESUS MARI|        26716.8|
|19134531c323f03ab...|JESUS MARI|        26716.8|
|8fd3ec468552fdda2...|JESUS MARI|        26716.8|
|2a0199bb8f20b9035...|JESUS MARI|        26716.8|
|2a0199bb8f20b9035...|JESUS MARI|        26716.8|
|2a0199bb8f20b9035...|JESUS MARI|        26716.8|
|2a0199bb8f20b9035...|JESUS MARI|        26716.8|
|8b23a5f47f37eaa52...|JESUS MARI|        26716.8|
|8b23a5f47f37eaa52...|JESUS MARI|        26716.8|
|8b23a5f47f37eaa52...|JESUS MARI|        26716.8|
|8b23a5f47f37eaa52...|JESUS MARI|        26716.8|
|8b23a5f47f37eaa52...|JESUS MARI|        26716.8|
|8b23a5f47f37eaa52...|JESUS MARI|        26716.8|
+--------------------+----------+---------------+
only showing top 15 rows

📊 Registros ordenados po

25/06/08 22:41:56 INFO CodeGenerator: Code generated in 7.025667 ms
25/06/08 22:41:56 INFO DAGScheduler: Registering RDD 110 (show at cmd16.sc:27) as input to shuffle 15
25/06/08 22:41:56 INFO DAGScheduler: Got map stage job 32 (show at cmd16.sc:27) with 1 output partitions
25/06/08 22:41:56 INFO DAGScheduler: Final stage: ShuffleMapStage 50 (show at cmd16.sc:27)
25/06/08 22:41:56 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:56 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:56 INFO DAGScheduler: Submitting ShuffleMapStage 50 (MapPartitionsRDD[110] at show at cmd16.sc:27), which has no missing parents
25/06/08 22:41:56 INFO MemoryStore: Block broadcast_32 stored as values in memory (estimated size 39.7 KiB, free 2004.5 MiB)
25/06/08 22:41:56 INFO MemoryStore: Block broadcast_32_piece0 stored as bytes in memory (estimated size 18.3 KiB, free 2004.5 MiB)
25/06/08 22:41:56 INFO BlockManagerInfo: Added broadcast_32_piece0 in memory on 192.168.18.245:57344 (siz

+----------+-------+------------+
|  distrito|periodo|monto_parque|
+----------+-------+------------+
|JESUS MARI|   2025|       284.4|
|JESUS MARI|   2025|      275.53|
|JESUS MARI|   2025|       272.4|
|JESUS MARI|   2025|      269.68|
|JESUS MARI|   2025|       264.8|
|JESUS MARI|   2025|      261.04|
|JESUS MARI|   2025|      260.71|
|JESUS MARI|   2025|      254.26|
|JESUS MARI|   2025|      250.92|
|JESUS MARI|   2025|      249.71|
|JESUS MARI|   2025|      249.68|
|JESUS MARI|   2025|      248.85|
|JESUS MARI|   2025|      248.07|
|JESUS MARI|   2025|      245.16|
|JESUS MARI|   2025|      239.71|
+----------+-------+------------+
only showing top 15 rows



ordenadoPorMonto: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, distrito: string ... 1 more field]
ordenMultiple: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, periodo: int ... 1 more field]

In [17]:
// 4. UTILIZAR GROUPBY Y COUNT
println("📊 4. UTILIZAR GROUPBY Y COUNT")

// GroupBy por distrito
val contadoPorDistrito = df.groupBy("distrito")
  .count()
  .orderBy(desc("count"))

println("📊 Conteo por distrito:")
contadoPorDistrito.show()

// GroupBy por período
val contadoPorPeriodo = df.groupBy("periodo")
  .count()
  .orderBy("periodo")

println("📊 Conteo por período:")
contadoPorPeriodo.show()

// GroupBy múltiple: distrito y período
val conteoPorDistritoPeriodo = df.groupBy("distrito", "periodo")
  .count()
  .filter(col("count") > 100)
  .orderBy(desc("count"))

println("📊 Conteo por distrito y período (más de 100 registros):")
conteoPorDistritoPeriodo.show()

📊 4. UTILIZAR GROUPBY Y COUNT
📊 Conteo por distrito:


25/06/08 22:41:57 INFO CodeGenerator: Code generated in 6.833958 ms
25/06/08 22:41:57 INFO DAGScheduler: Registering RDD 117 (show at cmd17.sc:10) as input to shuffle 16
25/06/08 22:41:57 INFO DAGScheduler: Got map stage job 34 (show at cmd17.sc:10) with 1 output partitions
25/06/08 22:41:57 INFO DAGScheduler: Final stage: ShuffleMapStage 53 (show at cmd17.sc:10)
25/06/08 22:41:57 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:57 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:57 INFO DAGScheduler: Submitting ShuffleMapStage 53 (MapPartitionsRDD[117] at show at cmd17.sc:10), which has no missing parents
25/06/08 22:41:57 INFO MemoryStore: Block broadcast_34 stored as values in memory (estimated size 34.9 KiB, free 2004.4 MiB)
25/06/08 22:41:57 INFO MemoryStore: Block broadcast_34_piece0 stored as bytes in memory (estimated size 16.5 KiB, free 2004.4 MiB)
25/06/08 22:41:57 INFO BlockManagerInfo: Added broadcast_34_piece0 in memory on 192.168.18.245:57344 (siz

+----------+------+
|  distrito| count|
+----------+------+
|JESUS MARI|220795|
|      NULL|    37|
+----------+------+

📊 Conteo por período:


25/06/08 22:41:57 INFO CodeGenerator: Code generated in 6.841708 ms
25/06/08 22:41:57 INFO DAGScheduler: Registering RDD 124 (show at cmd17.sc:18) as input to shuffle 17
25/06/08 22:41:57 INFO DAGScheduler: Got map stage job 36 (show at cmd17.sc:18) with 1 output partitions
25/06/08 22:41:57 INFO DAGScheduler: Final stage: ShuffleMapStage 56 (show at cmd17.sc:18)
25/06/08 22:41:57 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:57 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:57 INFO DAGScheduler: Submitting ShuffleMapStage 56 (MapPartitionsRDD[124] at show at cmd17.sc:18), which has no missing parents
25/06/08 22:41:57 INFO MemoryStore: Block broadcast_36 stored as values in memory (estimated size 34.5 KiB, free 2004.5 MiB)
25/06/08 22:41:57 INFO MemoryStore: Block broadcast_36_piece0 stored as bytes in memory (estimated size 16.3 KiB, free 2004.5 MiB)
25/06/08 22:41:57 INFO BlockManagerInfo: Added broadcast_36_piece0 in memory on 192.168.18.245:57344 (siz

+-------+-----+
|periodo|count|
+-------+-----+
|   2023|69183|
|   2024|73341|
|   2025|78308|
+-------+-----+

📊 Conteo por distrito y período (más de 100 registros):


25/06/08 22:41:57 INFO CodeGenerator: Code generated in 6.716291 ms
25/06/08 22:41:57 INFO DAGScheduler: Registering RDD 131 (show at cmd17.sc:27) as input to shuffle 18
25/06/08 22:41:57 INFO DAGScheduler: Got map stage job 38 (show at cmd17.sc:27) with 1 output partitions
25/06/08 22:41:57 INFO DAGScheduler: Final stage: ShuffleMapStage 59 (show at cmd17.sc:27)
25/06/08 22:41:57 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:57 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:57 INFO DAGScheduler: Submitting ShuffleMapStage 59 (MapPartitionsRDD[131] at show at cmd17.sc:27), which has no missing parents
25/06/08 22:41:57 INFO MemoryStore: Block broadcast_38 stored as values in memory (estimated size 36.2 KiB, free 2004.4 MiB)
25/06/08 22:41:57 INFO MemoryStore: Block broadcast_38_piece0 stored as bytes in memory (estimated size 17.0 KiB, free 2004.4 MiB)
25/06/08 22:41:57 INFO BlockManagerInfo: Added broadcast_38_piece0 in memory on 192.168.18.245:57344 (siz

+----------+-------+-----+
|  distrito|periodo|count|
+----------+-------+-----+
|JESUS MARI|   2025|78294|
|JESUS MARI|   2024|73329|
|JESUS MARI|   2023|69172|
+----------+-------+-----+



contadoPorDistrito: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, count: bigint]
contadoPorPeriodo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [periodo: int, count: bigint]
conteoPorDistritoPeriodo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, periodo: int ... 1 more field]

In [18]:
// 5. CONSULTA CON PROMEDIO
println("📊 5. CONSULTA CON PROMEDIO")

// Promedio de montos por distrito
val promediosPorDistrito = df.groupBy("distrito")
  .agg(
    avg(col("monto_serenazgo").cast("double")).as("promedio_serenazgo"),
    avg(col("monto_parque_jardin").cast("double")).as("promedio_parque"),
    avg(col("porcentaje_condominio").cast("double")).as("promedio_participacion"),
    count("*").as("total_registros")
  )
  .filter(col("total_registros") > 50)
  .orderBy(desc("promedio_serenazgo"))

println("📊 Promedios por distrito:")
promediosPorDistrito.show()

// Promedio por período
val promediosPorPeriodo = df.groupBy("periodo")
  .agg(
    round(avg(col("monto_residuos_solidos").cast("double")), 2).as("promedio_residuos"),
    round(avg(col("monto_barrido_calles").cast("double")), 2).as("promedio_barrido"),
    count("*").as("registros")
  )
  .orderBy("periodo")

println("📊 Promedios por período:")
promediosPorPeriodo.show()

📊 5. CONSULTA CON PROMEDIO
📊 Promedios por distrito:


25/06/08 22:41:58 INFO CodeGenerator: Code generated in 8.283625 ms
25/06/08 22:41:58 INFO DAGScheduler: Registering RDD 138 (show at cmd18.sc:16) as input to shuffle 19
25/06/08 22:41:58 INFO DAGScheduler: Got map stage job 40 (show at cmd18.sc:16) with 1 output partitions
25/06/08 22:41:58 INFO DAGScheduler: Final stage: ShuffleMapStage 62 (show at cmd18.sc:16)
25/06/08 22:41:58 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:58 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:58 INFO DAGScheduler: Submitting ShuffleMapStage 62 (MapPartitionsRDD[138] at show at cmd18.sc:16), which has no missing parents
25/06/08 22:41:58 INFO MemoryStore: Block broadcast_40 stored as values in memory (estimated size 50.3 KiB, free 2004.3 MiB)
25/06/08 22:41:58 INFO MemoryStore: Block broadcast_40_piece0 stored as bytes in memory (estimated size 21.5 KiB, free 2004.3 MiB)
25/06/08 22:41:58 INFO BlockManagerInfo: Added broadcast_40_piece0 in memory on 192.168.18.245:57344 (siz

+----------+------------------+------------------+----------------------+---------------+
|  distrito|promedio_serenazgo|   promedio_parque|promedio_participacion|total_registros|
+----------+------------------+------------------+----------------------+---------------+
|JESUS MARI|263.20437632220467|161.62826495171697|     83.28701863719598|         220795|
+----------+------------------+------------------+----------------------+---------------+

📊 Promedios por período:


25/06/08 22:41:58 INFO CodeGenerator: Code generated in 6.740375 ms
25/06/08 22:41:58 INFO DAGScheduler: Registering RDD 145 (show at cmd18.sc:28) as input to shuffle 20
25/06/08 22:41:58 INFO DAGScheduler: Got map stage job 42 (show at cmd18.sc:28) with 1 output partitions
25/06/08 22:41:58 INFO DAGScheduler: Final stage: ShuffleMapStage 65 (show at cmd18.sc:28)
25/06/08 22:41:58 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:58 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:58 INFO DAGScheduler: Submitting ShuffleMapStage 65 (MapPartitionsRDD[145] at show at cmd18.sc:28), which has no missing parents
25/06/08 22:41:58 INFO MemoryStore: Block broadcast_42 stored as values in memory (estimated size 45.4 KiB, free 2004.1 MiB)
25/06/08 22:41:58 INFO MemoryStore: Block broadcast_42_piece0 stored as bytes in memory (estimated size 20.3 KiB, free 2004.1 MiB)
25/06/08 22:41:58 INFO BlockManagerInfo: Added broadcast_42_piece0 in memory on 192.168.18.245:57344 (siz

+-------+-----------------+----------------+---------+
|periodo|promedio_residuos|promedio_barrido|registros|
+-------+-----------------+----------------+---------+
|   2023|            61.12|           26.52|    69183|
|   2024|            54.25|           27.08|    73341|
|   2025|             58.7|            29.4|    78308|
+-------+-----------------+----------------+---------+



promediosPorDistrito: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, promedio_serenazgo: double ... 3 more fields]
promediosPorPeriodo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [periodo: int, promedio_residuos: double ... 2 more fields]

In [19]:
// 6. TRES CONSULTAS CON JOIN
println("🔗 6. TRES CONSULTAS CON JOIN")

// Crear DataFrames derivados para hacer JOIN
// JOIN 1: Unir contribuyentes con sus totales por distrito
val totalPorContribuyente = df.groupBy("cod_contribuyente", "distrito")
  .agg(
    sum(col("monto_serenazgo").cast("double")).as("total_serenazgo"),
    count("*").as("num_predios")
  )
  .filter(col("num_predios") > 1)

val promedioDistrito = df.groupBy("distrito")
  .agg(
    avg(col("monto_serenazgo").cast("double")).as("promedio_distrito_serenazgo")
  )

val join1 = totalPorContribuyente
  .join(promedioDistrito, Seq("distrito"), "inner")
  .withColumn("diferencia_promedio", 
              round(col("total_serenazgo") - col("promedio_distrito_serenazgo"), 2))
  .orderBy(desc("diferencia_promedio"))

println("📊 JOIN 1 - Contribuyentes vs Promedio de distrito:")
join1.show(10)

// JOIN 2: Contribuyentes con múltiples períodos
val contribuyentesPeriodos = df.groupBy("cod_contribuyente")
  .agg(
    countDistinct("periodo").as("num_periodos"),
    collect_list("periodo").as("periodos")
  )
  .filter(col("num_periodos") > 1)

val datosContribuyente = df.select("cod_contribuyente", "distrito", "monto_parque_jardin")
  .distinct()

val join2 = contribuyentesPeriodos
  .join(datosContribuyente, Seq("cod_contribuyente"), "inner")
  .orderBy(desc("num_periodos"))

println("📊 JOIN 2 - Contribuyentes con múltiples períodos:")
join2.show(10)

// JOIN 3: Self-join para comparar períodos
val periodo2024 = df.filter(col("periodo") === "2024")
  .select(
    col("cod_contribuyente").as("contribuyente_2024"),
    col("distrito").as("distrito_2024"),
    col("monto_serenazgo").cast("double").as("monto_2024")
  )

val periodo2025 = df.filter(col("periodo") === "2025")
  .select(
    col("cod_contribuyente").as("contribuyente_2025"),
    col("distrito").as("distrito_2025"),
    col("monto_serenazgo").cast("double").as("monto_2025")
  )

val join3 = periodo2024
  .join(periodo2025, 
        col("contribuyente_2024") === col("contribuyente_2025") && 
        col("distrito_2024") === col("distrito_2025"), 
        "inner")
  .withColumn("variacion_monto", 
              round(col("monto_2025") - col("monto_2024"), 2))
  .orderBy(desc("variacion_monto"))

println("📊 JOIN 3 - Comparación entre períodos 2024-2025:")
join3.show(10)

🔗 6. TRES CONSULTAS CON JOIN
📊 JOIN 1 - Contribuyentes vs Promedio de distrito:


25/06/08 22:41:58 INFO CodeGenerator: Code generated in 7.318875 ms
25/06/08 22:41:58 INFO DAGScheduler: Registering RDD 153 (show at cmd19.sc:25) as input to shuffle 21
25/06/08 22:41:58 INFO DAGScheduler: Got map stage job 44 (show at cmd19.sc:25) with 1 output partitions
25/06/08 22:41:58 INFO DAGScheduler: Final stage: ShuffleMapStage 68 (show at cmd19.sc:25)
25/06/08 22:41:58 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:58 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:58 INFO DAGScheduler: Submitting ShuffleMapStage 68 (MapPartitionsRDD[153] at show at cmd19.sc:25), which has no missing parents
25/06/08 22:41:58 INFO MemoryStore: Block broadcast_44 stored as values in memory (estimated size 42.6 KiB, free 2004.4 MiB)
25/06/08 22:41:58 INFO MemoryStore: Block broadcast_44_piece0 stored as bytes in memory (estimated size 19.7 KiB, free 2004.4 MiB)
25/06/08 22:41:58 INFO BlockManagerInfo: Added broadcast_44_piece0 in memory on 192.168.18.245:57344 (siz

+----------+--------------------+------------------+-----------+---------------------------+-------------------+
|  distrito|   cod_contribuyente|   total_serenazgo|num_predios|promedio_distrito_serenazgo|diferencia_promedio|
+----------+--------------------+------------------+-----------+---------------------------+-------------------+
|JESUS MARI|8b23a5f47f37eaa52...| 709412.4000000006|         33|          263.2043763221368|           709149.2|
|JESUS MARI|55351ada81c0f27f8...| 657369.2400000077|        599|          263.2043763221368|          657106.04|
|JESUS MARI|2a0199bb8f20b9035...| 288995.5199999999|         19|          263.2043763221368|          288732.32|
|JESUS MARI|2119b2fb48bfa34d8...|286145.27999999997|         11|          263.2043763221368|          285882.08|
|JESUS MARI|9e17ffa1e26b269ea...| 283911.6000000009|        201|          263.2043763221368|           283648.4|
|JESUS MARI|e50b873fbaacca239...|         277559.04|         12|          263.2043763221368|    

25/06/08 22:41:59 INFO CodeGenerator: Code generated in 1.788875 ms
25/06/08 22:41:59 INFO DAGScheduler: Registering RDD 167 (show at cmd19.sc:43) as input to shuffle 23
25/06/08 22:41:59 INFO DAGScheduler: Got map stage job 48 (show at cmd19.sc:43) with 1 output partitions
25/06/08 22:41:59 INFO DAGScheduler: Final stage: ShuffleMapStage 74 (show at cmd19.sc:43)
25/06/08 22:41:59 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:41:59 INFO DAGScheduler: Missing parents: List()
25/06/08 22:41:59 INFO DAGScheduler: Submitting ShuffleMapStage 74 (MapPartitionsRDD[167] at show at cmd19.sc:43), which has no missing parents
25/06/08 22:41:59 INFO MemoryStore: Block broadcast_49 stored as values in memory (estimated size 26.0 KiB, free 1996.5 MiB)
25/06/08 22:41:59 INFO MemoryStore: Block broadcast_49_piece0 stored as bytes in memory (estimated size 12.7 KiB, free 1996.5 MiB)
25/06/08 22:41:59 INFO BlockManagerInfo: Added broadcast_49_piece0 in memory on 192.168.18.245:57344 (siz

+--------------------+------------+--------------------+----------+-------------------+
|   cod_contribuyente|num_periodos|            periodos|  distrito|monto_parque_jardin|
+--------------------+------------+--------------------+----------+-------------------+
|00193f7b6fa1046bd...|           3|  [2023, 2025, 2024]|JESUS MARI|             272.40|
|00272324be15fed92...|           3|  [2024, 2023, 2025]|JESUS MARI|             273.00|
|002edb71726eeee06...|           3|  [2023, 2025, 2024]|JESUS MARI|             272.40|
|00272324be15fed92...|           3|  [2024, 2023, 2025]|JESUS MARI|             282.00|
|00058fda87076ff60...|           3|[2024, 2024, 2023...|JESUS MARI|               0.00|
|00272324be15fed92...|           3|  [2024, 2023, 2025]|JESUS MARI|             250.92|
|0053b08719900f22c...|           3|[2023, 2023, 2024...|JESUS MARI|             272.40|
|0046a5cb0241f5c03...|           3|[2025, 2025, 2023...|JESUS MARI|             250.92|
|0001d6f5871abfe10...|          

25/06/08 22:42:00 INFO CodeGenerator: Code generated in 1.992959 ms
25/06/08 22:42:00 INFO DAGScheduler: Registering RDD 189 (show at cmd19.sc:70) as input to shuffle 27
25/06/08 22:42:00 INFO DAGScheduler: Got map stage job 53 (show at cmd19.sc:70) with 1 output partitions
25/06/08 22:42:00 INFO DAGScheduler: Final stage: ShuffleMapStage 85 (show at cmd19.sc:70)
25/06/08 22:42:00 INFO DAGScheduler: Parents of final stage: List()
25/06/08 22:42:00 INFO DAGScheduler: Missing parents: List()
25/06/08 22:42:00 INFO DAGScheduler: Submitting ShuffleMapStage 85 (MapPartitionsRDD[189] at show at cmd19.sc:70), which has no missing parents
25/06/08 22:42:00 INFO MemoryStore: Block broadcast_54 stored as values in memory (estimated size 15.9 KiB, free 2004.5 MiB)
25/06/08 22:42:00 INFO MemoryStore: Block broadcast_54_piece0 stored as bytes in memory (estimated size 8.1 KiB, free 2004.5 MiB)
25/06/08 22:42:00 INFO BlockManagerInfo: Added broadcast_54_piece0 in memory on 192.168.18.245:57344 (size

+--------------------+-------------+----------+--------------------+-------------+----------+---------------+
|  contribuyente_2024|distrito_2024|monto_2024|  contribuyente_2025|distrito_2025|monto_2025|variacion_monto|
+--------------------+-------------+----------+--------------------+-------------+----------+---------------+
|61dfe35d8fa6fc29d...|   JESUS MARI|    331.56|61dfe35d8fa6fc29d...|   JESUS MARI|   26716.8|       26385.24|
|61dfe35d8fa6fc29d...|   JESUS MARI|    331.56|61dfe35d8fa6fc29d...|   JESUS MARI|   26716.8|       26385.24|
|61dfe35d8fa6fc29d...|   JESUS MARI|    331.56|61dfe35d8fa6fc29d...|   JESUS MARI|   26716.8|       26385.24|
|61dfe35d8fa6fc29d...|   JESUS MARI|    331.56|61dfe35d8fa6fc29d...|   JESUS MARI|   26716.8|       26385.24|
|8b23a5f47f37eaa52...|   JESUS MARI|    915.48|8b23a5f47f37eaa52...|   JESUS MARI|   26716.8|       25801.32|
|8b23a5f47f37eaa52...|   JESUS MARI|    915.48|8b23a5f47f37eaa52...|   JESUS MARI|   26716.8|       25801.32|
|8b23a5f47

totalPorContribuyente: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, distrito: string ... 2 more fields]
promedioDistrito: DataFrame = [distrito: string, promedio_distrito_serenazgo: double]
join1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [distrito: string, cod_contribuyente: string ... 4 more fields]
contribuyentesPeriodos: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, num_periodos: bigint ... 1 more field]
datosContribuyente: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, distrito: string ... 1 more field]
join2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cod_contribuyente: string, num_periodos: bigint ... 3 more fields]
periodo2024: DataFrame = [contribuyente_2024: string, distrito_2024: string ... 1 more field]
periodo2025: DataFrame = [contribuyente_2025: string, distrito_2025: string ... 1 more field]
join3: org.apache.spark.sql.Datase